# Maltese PQ Synthetic Data Generator (Google Gemini API)

This notebook replicates the exercise you ran in ChatGPT:

- Generate **10 synthetic rows** for each **question type**: **A1, A2, A3**
- Generate **10 synthetic rows** for each **answer type**: **B1, B2, B3**
- Generate **one set at a time** (you run the next cell when you want the next set)

The output is **synthetic** (fabricated but plausible) and shaped to match the traits/format of the sample set you provided.

## Prerequisites
You need a **Google AI Studio API key**.

Set these environment variables (recommended), or fill them in directly below:

- `GOOGLE_API_KEY`
- `GEMINI_MODEL` (optional, default: `gemini-flash-latest`)

Optionally, place them in a local `.env` file.

### Plain English
This notebook uses Google Gemini to create synthetic Maltese parliamentary question/answer records by category and save them as CSV files.


In [1]:
# Install the Python packages this notebook depends on.
# Run this once in a fresh environment.

# If needed, install dependencies
%pip -q install pandas python-dotenv requests

Note: you may need to restart the kernel to use updated packages.


In [4]:
# Import required libraries, load .env values, validate GOOGLE_API_KEY,
# and prepare the Gemini API URL used by the helper functions.

import os
import json
import re
import time
from dataclasses import dataclass
from typing import Dict, List, Any

import pandas as pd
from dotenv import load_dotenv

import requests

load_dotenv()  # loads .env if present

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
GEMINI_MODEL = os.getenv("GEMINI_MODEL", "gemini-flash-latest")

missing = [k for k,v in {
    "GOOGLE_API_KEY": GOOGLE_API_KEY,
}.items() if not v]

if missing:
    raise ValueError(
        "Missing required environment variables: " + ", ".join(missing) + "\n"
        "Set them in your shell or create a .env file."
    )

GEMINI_API_URL = f"https://generativelanguage.googleapis.com/v1beta/models/{GEMINI_MODEL}:generateContent"

print(f"Gemini API configured. Model: {GEMINI_MODEL}")

Gemini API configured. Model: gemini-flash-latest


## Category specs

- **A1**: Information-seeking questions
- **A2**: Assertion / challenge questions (hidden accusation)
- **A3**: Request / directive questions
- **B1**: Replies (direct answers)
- **B2**: Answers by implication (partial/indirect)
- **B3**: Non-replies (defer/referral/reroute)

### Column schemas
- For **A1–A3**: `Date | PQ No. | MP | Ministry (EN) | Title (EN) | Question (EN) | Answer (EN)`
- For **B1–B3**: `Date | PQ No. | MP | Question (EN) | Answer (EN)`

### Plain English
This section defines what each category means and what columns each generated table must contain.


In [5]:
# Define category rules, output column schemas, and allowed MPs/ministries
# so generated rows stay realistic and consistent.

@dataclass(frozen=True)
class CategorySpec:
    id: str
    name: str
    kind: str  # 'question' or 'answer'
    columns: List[str]
    instruction: str


QUESTION_COLUMNS = [
    "Date", "PQ No.", "MP", "Ministry (EN)", "Title (EN)", "Question (EN)", "Answer (EN)"
]
ANSWER_COLUMNS = [
    "Date", "PQ No.", "MP", "Question (EN)", "Answer (EN)"
]

CATEGORY_SPECS: Dict[str, CategorySpec] = {
    "A1": CategorySpec(
        id="A1",
        name="Information-Seeking Questions",
        kind="question",
        columns=QUESTION_COLUMNS,
        instruction=(
            "Generate information-seeking parliamentary questions. The MP is primarily asking for missing facts "
            "(counts, dates, locations, status). Keep it neutral and specific."
        ),
    ),
    "A2": CategorySpec(
        id="A2",
        name="Assertion / Hidden-Accusation Questions",
        kind="question",
        columns=QUESTION_COLUMNS,
        instruction=(
            "Generate assertion/challenge parliamentary questions with a hidden accusation or accountability push. "
            "The wording should imply criticism (delay, lack of transparency, disruption, poor performance) while still being phrased as a question."
        ),
    ),
    "A3": CategorySpec(
        id="A3",
        name="Request / Directive Questions",
        kind="question",
        columns=QUESTION_COLUMNS,
        instruction=(
            "Generate directive parliamentary questions that ask the Minister/PM to do something: lay documents on the Table, provide lists, "
            "provide a site plan/timeframe, confirm a contract, publish a breakdown, etc."
        ),
    ),
    "B1": CategorySpec(
        id="B1",
        name="Replies (Direct Answers)",
        kind="answer",
        columns=ANSWER_COLUMNS,
        instruction=(
            "Generate answers that directly address the question with the requested facts or a clear yes/no plus details. "
            "Avoid deferrals and avoid referrals."
        ),
    ),
    "B2": CategorySpec(
        id="B2",
        name="Answers by Implication (Indirect/Partial)",
        kind="answer",
        columns=ANSWER_COLUMNS,
        instruction=(
            "Generate answers that are partial/indirect: give an update, a condition, or a vague timeline instead of the exact requested item. "
            "Examples: 'being processed', 'in the coming months', 'depends on certification', 'will be provided in a subsequent sitting'."
        ),
    ),
    "B3": CategorySpec(
        id="B3",
        name="Non-Replies (Deferral/Referral/Reroute)",
        kind="answer",
        columns=ANSWER_COLUMNS,
        instruction=(
            "Generate non-replies: defer to a future sitting, refer to another PQ, or reroute to another minister. "
            "Do not provide the substance of the requested info."
        ),
    ),
}

MPS = [
    "Ivan Bartolo",
    "Chris Said",
    "Justin Schembri",
    "Jerome Caruana Cilia",
    "Graziella Attard Previ",
    "Rebekah Borg",
    "Ivan Castillo",
]

MINISTRIES = [
    "Prime Minister",
    "Justice and Reform of the Construction Sector",
    "Environment, Energy and Regeneration of the Grand Harbour",
    "Education, Sport, Youth, Research and Innovation",
    "Home Affairs, Security, Reforms and Equality",
    "Social Policy and Children’s Rights",
    "Health and Active Ageing",
    "Economy, European Funds and Lands",
    "Transport, Infrastructure and Public Works",
    "Gozo",
]

def _today_ddmmyyyy() -> str:
    # Notebook runs on your local machine; we don't assume timezone here.
    from datetime import datetime
    return datetime.now().strftime("%d/%m/%Y")

print("Loaded category specs:", ", ".join(CATEGORY_SPECS.keys()))

Loaded category specs: A1, A2, A3, B1, B2, B3


## LLM prompt + generation helpers

We request **JSON-only output** (array of objects). Then we validate:

- Exactly **N** rows
- Exact **column names**
- Date format: `DD/MM/YYYY`
- PQ No.: numeric string/int

If validation fails, we retry once with a corrective instruction.

### Plain English
This section explains the helper functions that call Gemini, force JSON output, and validate structure before saving results.


In [6]:
# This is the core generation pipeline:
# 1) build prompt text, 2) call Gemini, 3) parse JSON safely,
# 4) validate rows, and 5) retry with corrections if needed.

DATE_RE = re.compile(r"^\d{2}/\d{2}/\d{4}$")

SYSTEM_STYLE = (
    "You generate synthetic Maltese parliamentary question (PQ) records in English. "
    "They must be fabricated but plausible, using realistic ministry names and parliamentary phrasing. "
    "Do NOT use real parliamentary records or real events; do NOT copy any real PQ verbatim. "
    "Keep the tone formal and consistent with parliamentary replies."
)

def _messages_to_gemini_prompt(messages: List[Dict[str, str]]) -> str:
    chunks = []
    for m in messages:
        role = (m.get("role") or "user").upper()
        content = m.get("content", "")
        chunks.append(f"{role}:\n{content}")
    chunks.append("Return valid JSON only. Do not include markdown fences.")
    return "\n\n".join(chunks)


def _extract_json_from_text(text: str) -> Any:
    cleaned = text.strip()
    if cleaned.startswith("```"):
        cleaned = re.sub(r"^```(?:json)?\\s*", "", cleaned, flags=re.IGNORECASE)
        cleaned = re.sub(r"\\s*```$", "", cleaned)

    try:
        return json.loads(cleaned)
    except json.JSONDecodeError:
        # Recover if model prepends/appends non-JSON text.
        starts = [i for i in (cleaned.find("["), cleaned.find("{")) if i != -1]
        end = max(cleaned.rfind("]"), cleaned.rfind("}"))
        if not starts or end == -1:
            raise
        snippet = cleaned[min(starts):end + 1]
        return json.loads(snippet)


def call_gemini_chat_json(messages: List[Dict[str, str]], temperature: float = 0.7, max_tokens: int = 2000) -> Any:
    """Call Gemini generateContent API and parse JSON from the model response."""
    prompt = _messages_to_gemini_prompt(messages)
    payload = {
        "contents": [{"parts": [{"text": prompt}]}],
        "generationConfig": {
            "temperature": temperature,
            "maxOutputTokens": max_tokens,
            "responseMimeType": "application/json",
        },
    }

    resp = requests.post(
        f"{GEMINI_API_URL}?key={GOOGLE_API_KEY}",
        headers={"Content-Type": "application/json"},
        json=payload,
        timeout=120,
    )
    resp.raise_for_status()
    data = resp.json()

    candidates = data.get("candidates") or []
    if not candidates:
        raise ValueError(f"Gemini returned no candidates: {data}")

    parts = candidates[0].get("content", {}).get("parts", [])
    text = "".join(p.get("text", "") for p in parts if isinstance(p, dict)).strip()
    if not text:
        raise ValueError(f"Gemini response had no text content: {data}")

    parsed = _extract_json_from_text(text)
    if isinstance(parsed, dict) and "rows" in parsed:
        return parsed["rows"]
    return parsed


def validate_rows(rows: List[Dict[str, Any]], columns: List[str], n: int) -> List[str]:
    errs = []
    if not isinstance(rows, list):
        return ["Output is not a JSON array."]
    if len(rows) != n:
        errs.append(f"Expected {n} rows, got {len(rows)}.")
    for i, r in enumerate(rows):
        if not isinstance(r, dict):
            errs.append(f"Row {i} is not an object.")
            continue
        missing = [c for c in columns if c not in r]
        extra = [k for k in r.keys() if k not in columns]
        if missing:
            errs.append(f"Row {i} missing columns: {missing}")
        if extra:
            errs.append(f"Row {i} has extra columns: {extra}")
        if "Date" in r and isinstance(r.get("Date"), str) and not DATE_RE.match(r["Date"]):
            errs.append(f"Row {i} has invalid Date format: {r.get('Date')}")
        if "PQ No." in r:
            pq = str(r.get("PQ No."))
            if not pq.isdigit():
                errs.append(f"Row {i} has non-numeric PQ No.: {r.get('PQ No.')}")
    return errs


def generate_set(category_id: str, n: int = 10, temperature: float = 0.7, retries: int = 1) -> pd.DataFrame:
    if category_id not in CATEGORY_SPECS:
        raise KeyError(f"Unknown category_id: {category_id}. Use one of {list(CATEGORY_SPECS.keys())}")

    spec = CATEGORY_SPECS[category_id]
    columns = spec.columns

    base_user_prompt = {
        "role": "user",
        "content": (
            f"Create {n} synthetic Maltese PQ-style records for category {spec.id}: {spec.name}.\n\n"
            f"Category instruction: {spec.instruction}\n\n"
            "Hard requirements:\n"
            f"- Output MUST be valid JSON. Prefer a bare JSON array of objects.\n"
            f"- Each object MUST have exactly these keys: {columns}.\n"
            "- Date must be DD/MM/YYYY.\n"
            "- PQ No. must be numeric. Use plausible ranges (e.g., 10000–40000).\n"
            f"- MP must be one of: {MPS}.\n"
            f"- If the schema includes Ministry (EN), it must be one of: {MINISTRIES}.\n"
            "- Titles should be short and look like Maltese PQ titles (topic – detail).\n"
            "- Questions and answers must be formal parliamentary English.\n"
            "- Content must be fictional (no real cases, no real named projects beyond generic labels).\n"
            "- Do not add commentary or markdown—JSON only.\n"
        )
    }

    messages = [
        {"role": "system", "content": SYSTEM_STYLE},
        base_user_prompt,
    ]

    last_errs: List[str] = []
    for attempt in range(retries + 1):
        rows = call_gemini_chat_json(messages, temperature=temperature, max_tokens=2500)
        errs = validate_rows(rows, columns, n)
        if not errs:
            df = pd.DataFrame(rows, columns=columns)
            return df

        last_errs = errs
        # Add a corrective instruction for the retry
        messages.append({
            "role": "user",
            "content": (
                "The previous output failed validation. Fix it and output JSON again. "
                "Do not change the schema. Ensure exactly the required keys and exactly N rows. "
                "Errors were:\n- " + "\n- ".join(errs)
            )
        })
        time.sleep(0.2)

    raise ValueError("Failed to generate a valid set. Validation errors:\n- " + "\n- ".join(last_errs))


## Run one set at a time

Run the cell below for a single category. Repeat for the next category when you're ready.

Valid category IDs: `A1, A2, A3, B1, B2, B3`.

### Plain English
This section tells you to generate one category at a time so you can inspect each output before moving on.


In [ ]:
# Create an output folder and define a helper that generates one category,
# saves it to CSV, and displays the result table.

OUTPUT_DIR = "pq_synthetic_outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)

def run_and_save(category_id: str, n: int = 10, temperature: float = 0.7) -> pd.DataFrame:
    df = generate_set(category_id=category_id, n=n, temperature=temperature, retries=1)
    stamp = time.strftime("%Y%m%d_%H%M%S")
    path = os.path.join(OUTPUT_DIR, f"{category_id}_{stamp}.csv")
    df.to_csv(path, index=False)
    display(df)
    print(f"Saved: {path}")
    return df

# Example usage (uncomment one at a time):
# df_A1 = run_and_save("A1")
# df_A2 = run_and_save("A2")
# df_A3 = run_and_save("A3")
# df_B1 = run_and_save("B1")
# df_B2 = run_and_save("B2")
# df_B3 = run_and_save("B3")

KeyboardInterrupt: 

## Optional: generate everything (still sequential)

If you want to generate all 6 sets in one go, run this cell.

### Plain English
This section provides an optional loop if you want all categories generated in one run.


In [ ]:
# Optional batch mode: generate all categories one after another.

# sets = {}
# for cid in ["A1", "A2", "A3", "B1", "B2", "B3"]:
#     print("\n=== Generating", cid, CATEGORY_SPECS[cid].name, "===")
#     sets[cid] = run_and_save(cid)

# sets

## Notes

- If you see token limit errors, reduce `n`, reduce prompt verbosity, or lower `max_tokens` in `call_gemini_chat_json()`.
- If your key is restricted by API or referrer settings, update restrictions in Google AI Studio before running.

### Plain English
This section lists troubleshooting tips for token limits and API key restrictions.
